In [1]:
import cv2
import numpy as np

In [ ]:
def resize(image, width=None, height=None, show=False):
    if width is None and height is None:
        return image

    if width is None:
        r = height / image.shape[0]
        width = int(r * image.shape[1])
    elif height is None:
        r = width / image.shape[1]
        height = int(r * image.shape[0])

    resized = cv2.resize(image, (width, height), interpolation=cv2.INTER_AREA)

    if show:
        cv2.imshow('Image Resized By {0:.2f}'.format(r), resized)
        cv2.waitKey(0)

    return resized

In [11]:
def draw_corners(img, points):
    
    if len(points) == 4:
        p1= points[0]
        p2= points[1]
        p3= points[2]
        p4 = points[3]
        cv2.circle(img, tuple(p1), 6, (0, 0, 255), -1)
        cv2.circle(img, tuple(p2), 6, (0, 255, 0), -1)
        cv2.circle(img, tuple(p3), 6, (255, 0, 0), -1)
        cv2.circle(img, tuple(p4), 6, (0, 255, 255), -1)

    else:
        from random import sample
        for p in points:
            random_color = tuple(sample(range(0, 255), 3))
            cv2.circle(img, tuple(p), 6, random_color, -1)

    cv2.imshow("image", img)
    cv2.waitKey(0)

In [12]:
def get_corner_points(contour):
    
    peri = cv2.arcLength(contour, True)
    corners = cv2.approxPolyDP(contour, 0.02 * peri, True)


    return np.squeeze(corners)

In [13]:
def order_corner_points_clockwise(points):
   
    rect = np.zeros((4, 2), dtype="float32")

    axis_sum = np.sum(points, axis=1)
    rect[0] = points[np.argmin(axis_sum)]
    rect[2] = points[np.argmax(axis_sum)]

   
    axis_diff = np.diff(points, axis=1)
    rect[1] = points[np.argmin(axis_diff)]
    rect[3] = points[np.argmax(axis_diff)]

   
    return rect

In [14]:
def apply_top_view(image, pts):
    (tl, tr, br, bl) = pts

   
    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))

    
    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))

    
    dst = np.array([
        [0, 0],
        [maxWidth - 1, 0],
        [maxWidth - 1, maxHeight - 1],
        [0, maxHeight - 1]], dtype="float32")

    
    M = cv2.getPerspectiveTransform(pts, dst)
    warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))

    
    return warped

In [15]:
def make_page_upright(src):

    image = None
    if isinstance(src, str):  
        image = cv2.imread(src)
    else:  
        image = src

    original_height, original_width = image.shape[:2]
    new_height = 400

    
    h_ratio = original_height / new_height

    
    small_image = resize(image, height=new_height)

   
    w_ratio = original_width / small_image.shape[1]

    gray_small_image = cv2.cvtColor(small_image, cv2.COLOR_BGR2GRAY)
    blurred_gray_small_image = cv2.GaussianBlur(gray_small_image, (9, 9), 0)
    canny_blurred_gray_small_image = cv2.Canny(
        blurred_gray_small_image, 120, 240)

   
    copy = small_image.copy()
    cnts = cv2.findContours(canny_blurred_gray_small_image,
                            cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[-2]

    biggest_contour = sorted(cnts, key=cv2.contourArea, reverse=True)[0]

    
    unordered_corners = get_corner_points(biggest_contour)
   
    corners = order_corner_points_clockwise(unordered_corners)
    
    new_corners = np.zeros((4, 2), dtype="float32")
    for i in range(len(corners)):
        new_corners[i] = [corners[i][0] * w_ratio, corners[i][1] * h_ratio]

    return apply_top_view(image, new_corners)

In [2]:

image = cv2.imread("3.jpeg")

original_height, original_width = image.shape[:2]
new_height = 400
cv2.imshow("image", image)
cv2.waitKey(0)

-1

In [5]:
ratio = original_height / new_height


small_image = resize(image, height=new_height)
cv2.imshow("image", small_image)
cv2.waitKey(0)

-1

In [6]:
gray_small_image = cv2.cvtColor(small_image, cv2.COLOR_BGR2GRAY)
cv2.imshow("image", gray_small_image)
cv2.waitKey(0)

-1

In [7]:
blurred_gray_small_image = cv2.GaussianBlur(gray_small_image, (9, 9), 0)
cv2.imshow("image", blurred_gray_small_image)
cv2.waitKey(0)

-1

In [8]:
canny_blurred_gray_small_image = cv2.Canny(blurred_gray_small_image, 120, 240)
cv2.imshow("image", canny_blurred_gray_small_image)
cv2.waitKey(0)

-1

In [9]:
copy = small_image.copy()
cnts = cv2.findContours(canny_blurred_gray_small_image,cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[-2]

biggest_contour = sorted(cnts, key=cv2.contourArea, reverse=True)[0]
cv2.drawContours(copy, [biggest_contour], -1, (0, 255, 0), 2)
cv2.imshow("image", copy)
cv2.waitKey(0)

-1

In [16]:
unordered_corners = get_corner_points(biggest_contour)
print(len(unordered_corners))
draw_corners(copy.copy(), unordered_corners)


corners = order_corner_points_clockwise(unordered_corners)
draw_corners(copy, np.int_(corners))

4


In [17]:
new_image = apply_top_view(image, np.float32(corners)*ratio)
new_image=cv2.rotate(new_image, cv2.ROTATE_180)
cv2.imshow("image", resize(new_image, height=400))
cv2.waitKey(0)

-1